In [7]:
import h2o
import pandas as pd
from h2o.automl import H2OAutoML

h2o.init()

# Assuming 'dataset.csv' is your dataset with movies
movies = pd.read_csv('ratings.csv')

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,2 hours 48 mins
H2O_cluster_timezone:,Asia/Colombo
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.3
H2O_cluster_version_age:,1 month and 29 days
H2O_cluster_name:,H2O_from_python_lenovo_33kb5q
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,2.856 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [8]:
movies = movies[['id', 'title', 'overview', 'genre']]
movies['tags'] = movies['overview'].fillna('') + movies['genre'].fillna('')
new_data = movies.drop(columns=['overview', 'genre'])

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import h2o
from h2o.automl import H2OAutoML
import pickle

In [11]:
# Feature extraction with CountVectorizer
cv = CountVectorizer(max_features=10000, stop_words='english')
vector = cv.fit_transform(new_data['tags'].values.astype('U')).toarray()

In [12]:
# Compute cosine similarity
similarity = cosine_similarity(vector)


In [13]:
def recommend(movie_title):
    index = new_data[new_data['title'] == movie_title].index[0]
    distances = sorted(list(enumerate(similarity[index])), reverse=True, key=lambda x: x[1])
    for i in distances[1:6]:  # start from 1 to avoid recommending the same movie
        print(new_data.iloc[i[0]].title)

In [14]:
recommend("Iron Man")

Iron Man 3
Guardians of the Galaxy Vol. 2
Avengers: Age of Ultron
Star Wars: Episode III - Revenge of the Sith
G.O.R.A.


In [15]:
# Saving necessary objects with pickle
pickle.dump(new_data, open('movies_list.pkl', 'wb'))
pickle.dump(similarity, open('similarity.pkl', 'wb'))